In [4]:
import networkx as nx
import pandas as pd

In [5]:
dirname = 'data/competition-output/phase2-final/rcc-03/raw_results/holdout/output/'
filename = 'data_set_citations.json'

In [6]:
df = pd.read_json(dirname+filename)
df['data_set_id'] = ['data_'+str(i) for i in df['data_set_id']]
df['publication_id'] = ['pub_'+str(i) for i in df['publication_id']]
df = df.drop('mention_list', axis=1)
df.head()

,data_set_id,publication_id,score
0,data_305,pub_103,0.264053
1,data_306,pub_103,0.429020
2,data_320,pub_103,0.374938
3,data_306,pub_104,0.293487
4,data_306,pub_106,0.343932


In [7]:
G = nx.Graph()
G.add_weighted_edges_from(df.values.tolist())

In [13]:
dirname = 'data/competition-output/phase2-final/rcc-03/raw_results/holdout/output/'
filename = 'research_fields.json'

In [14]:
df_rf = pd.read_json(dirname+filename)
# df_rf['data_set_id'] = ['data_'+str(i) for i in df_rf['data_set_id']]
df_rf['publication_id'] = ['pub_'+str(i) for i in df_rf['publication_id']]
# df_rf = df_rf.drop('mention_list', axis=1)
df_rf.head(15)

,publication_id,research_field,score
0,pub_102,business:finance,0.96
1,pub_103,economics:finance,0.82
2,pub_104,economics:finance,0.97
3,pub_106,economics:finance,0.97
4,pub_107,business:finance,0.91
5,pub_108,business:finance,0.76
6,pub_112,business:finance,0.79
7,pub_113,economics:finance,0.81
8,pub_114,sociology:finance,0.74
9,pub_117,economics:finance,0.77


In [15]:
set(df_rf.research_field)

{'business',
 'business:criminology',
 'business:education',
 'business:finance',
 'business:healthcare',
 'business:welfare',
 'economics',
 'economics:agriculture',
 'economics:criminology',
 'economics:education',
 'economics:finance',
 'economics:healthcare',
 'economics:welfare',
 'medicine:criminology',
 'medicine:education',
 'medicine:finance',
 'medicine:healthcare',
 'sociology',
 'sociology:agriculture',
 'sociology:criminology',
 'sociology:education',
 'sociology:finance',
 'sociology:healthcare',
 'sociology:welfare'}

In [20]:
from networkx.algorithms.connectivity import local_node_connectivity

## examples if it's cut off. 
local_node_connectivity(G, 'data_306', 'pub_103')

3